# mp4 파일 다운로드

In [3]:
import socket

client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client.bind(('', 45000))

n_h = int(client.recvfrom(1024)[0].decode())
n_d = int(client.recvfrom(1024)[0].decode())
data = b''

for i in range(n_h):
    data += client.recvfrom(65536)[0]

for i in range(n_d):
    data += client.recvfrom(65536)[0]
    open('/Users/winlab/Desktop/r_video.mp4','wb').write(data)
    
client.close()

# mp4 파일 다운로드 하면서 재생

In [ ]:
import socket
import numpy as np
import cv2
import threading
import time

client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
client.bind(('', 45000))

video_path = '/Users/winlab/Desktop/r_video.mp4'
n_h = int(client.recvfrom(1024)[0].decode())
n_d = int(client.recvfrom(1024)[0].decode())

data = b''
for i in range(n_h):
    data += client.recvfrom(65536)[0]
open(video_path,'wb').write(data)

def recv():
    for i in range(n_d):
        open(video_path,'ab').write(client.recvfrom(65536)[0])
        
t = threading.Thread(target = recv)
t.start()

index = 0
cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

while 1:
    video = cv2.VideoCapture(video_path)
    
    if video.get(5) == 0:
        time.sleep(0.1)
        continue
        
    f = round(1/video.get(5)*1000)
    
    video.set(1, index)
    while 1:
        ret, frame = video.read()
        if ret:
            cv2.imshow('Video', frame)
            cv2.waitKey(f-4)
        else:
            index = video.get(1)
            break
            
    if index != video.get(7):
        pass
    else:
        break
    
    video.release()

cv2.destroyAllWindows()
client.close()

# mp4 파일 다운로드 + 인덱싱 하면서 재생

In [1]:
import socket
import numpy as np
import cv2
import threading
import time

def indexing_mp4(v1_data, v_path2, count):
    with open(v_path2, 'rb') as v2: # 두번째 영상 바이너리 형식으로 읽어오기
        v2_data = v2.read()
    
    data_idx = v1_data.find(b'mdat')
    header_idx = v1_data.find(b'moov')
    
    if data_idx < header_idx:
        print("can't stream")
        return
    
    data_idx = v2_data.find(b'mdat')
    header_idx = v2_data.find(b'moov')
    
    if data_idx < header_idx:
        print("can't stream")
        return
    
    np_v1 = np.frombuffer(v1_data, dtype='uint8') # 바이너리 해석
    np_v2 = np.frombuffer(v2_data, dtype='uint8')
    np_v2 = np_v2[count:]

    l1 = len(np_v1)
    l2 = len(np_v2)
    if l2 < l1:
        xor_len = l2             # 데이터의 길이를 비교
        add_data = np_v1[l2:l1]
    else:
        xor_len = l1
        add_data = np_v2[l1:l2]
    
    xor_v_data = np.bitwise_xor(np_v1[:xor_len], np_v2[:xor_len]) # xor 연산

    xor_v_bin = xor_v_data.tostring() # 택 데이터 바이너리화
    return xor_v_bin

client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)  # 소켓 정의
client.bind(('', 45000))

n_h = int(client.recvfrom(1024)[0].decode())  # 헤더 분열 갯수
n_d = int(client.recvfrom(1024)[0].decode())  # 데이터 분열 갯수
header_data = b''
original_path = '/Users/winlab/Desktop/Yes.mp4'  # 가지고 있는 동영상의 경로
stream_path = '/Users/winlab/Desktop/stream.mp4'  # 저장할 경로와 이름

count = 0

for i in range(n_h):  # 헤더 받아서 인덱싱
    index_data = client.recvfrom(65536)[0]
    header_data += indexing_mp4(index_data, original_path, count)
    count += len(index_data)
open(stream_path,'wb').write(header_data)

def recv():  # 데이터 받아서 인덱싱
    global count
    for i in range(n_d):
        data = indexing_mp4(client.recvfrom(65536)[0], original_path, count)
        open(stream_path, 'ab').write(data)
        count += len(data)
        
t = threading.Thread(target = recv)
t.start()

index = 0
cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

while 1:
    video = cv2.VideoCapture(stream_path)
    
    if video.get(5) == 0:  # 받은 데이터가 없다면 잠시 쉬었다가 다시 실행
        time.sleep(0.1)
        continue
        
    f = round(1/video.get(5)*1000) - 3  # 초탕 프레임으로 waitkey값 계산
    
    video.set(1, index)  # 프레임 위치 설정
    while 1:
        ret, frame = video.read()
        if ret:
            cv2.imshow('Video', frame)
            cv2.waitKey(f)
        else:
            index = video.get(1)  # 프레임 위치 저장
            break
            
    if index != video.get(7):  # 총 프레임수 만큼 재생했다면 종료
        pass
    else:
        break
    
    video.release()

cv2.destroyAllWindows()
client.close()

In [1]:
import socket
import numpy as np
import cv2
import threading
import time

def indexing_mp4(v1_data, v_path2, count):
    with open(v_path2, 'rb') as v2: # 두번째 영상 바이너리 형식으로 읽어오기
        v2_data = v2.read()
    
    data_idx = v1_data.find(b'mdat')
    header_idx = v1_data.find(b'moov')
    
    if data_idx < header_idx:
        print("can't stream")
        return
    
    data_idx = v2_data.find(b'mdat')
    header_idx = v2_data.find(b'moov')
    
    if data_idx < header_idx:
        print("can't stream")
        return
    
    np_v1 = np.frombuffer(v1_data, dtype='uint8') # 바이너리 해석
    np_v2 = np.frombuffer(v2_data, dtype='uint8')
    np_v2 = np_v2[count:]

    l1 = len(np_v1)
    l2 = len(np_v2)
    if l2 < l1:
        xor_len = l2             # 데이터의 길이를 비교
        add_data = np_v1[l2:l1]
    else:
        xor_len = l1
        add_data = np_v2[l1:l2]
    
    xor_v_data = np.bitwise_xor(np_v1[:xor_len], np_v2[:xor_len]) # xor 연산

    xor_v_bin = xor_v_data.tostring() # 택 데이터 바이너리화
    return xor_v_bin

client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
client.bind(('', 45000))

n_h = int(client.recvfrom(1024)[0].decode())
n_d = int(client.recvfrom(1024)[0].decode())
header_data = b''
original_path = '/Users/winlab/Desktop/Yes.mp4'
stream_path = '/Users/winlab/Desktop/stream.mp4'

count = 0

for i in range(n_h):
    index_data = client.recvfrom(65536)[0]
    header_data += indexing_mp4(index_data, original_path, count)
    count += len(index_data)
open(stream_path,'wb').write(header_data)

for i in range(n_d):
    data = indexing_mp4(client.recvfrom(65536)[0], original_path, count)
    open(stream_path, 'ab').write(data)
    count += len(data)

0.031250715255737305
0.015604734420776367
0.046874284744262695
0.2461237907409668
0.20465350151062012
0.19718503952026367
0.20259547233581543
0.20746994018554688
0.19671082496643066
0.19027280807495117
0.20356130599975586
0.3316919803619385
0.0728144645690918
0.17984962463378906
0.2126145362854004
0.24115753173828125
0.3393583297729492
0.08004331588745117
0.3716762065887451
0.05197930335998535
0.1704864501953125
0.3043043613433838
0.11781954765319824
0.307462215423584
0.09020781517028809
0.20377874374389648
0.25220465660095215
0.18126869201660156
0.33182716369628906
0.08466124534606934
0.19317984580993652
0.2300422191619873
0.16881418228149414
0.3321833610534668
0.08467435836791992
0.20173954963684082
0.2204124927520752
0.21939992904663086
0.27415895462036133
0.13092613220214844
0.1862807273864746
0.23912358283996582
0.19508910179138184
0.21529364585876465
0.20061111450195312
0.33226823806762695
0.06801962852478027
0.1942300796508789
0.23589062690734863
0.1821143627166748
0.33741188049

PermissionError: [Errno 13] Permission denied: '/Users/winlab/Desktop/stream.mp4'

In [ ]:
import cv2
index = 0
cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

while 1:
    video = cv2.VideoCapture('/Users/winlab/Desktop/stream.mp4')
    
    if video.get(5) == 0:
        time.sleep(0.1)
        continue
        
    f = round(1/video.get(5)*1000)
    
    video.set(1, index)
    while 1:
        ret, frame = video.read()
        if ret:
            cv2.imshow('Video', frame)
            cv2.waitKey(f-4)
        else:
            index = video.get(1)
            break
            
    if index != video.get(7):
        pass
    else:
        break
    
    video.release()

cv2.destroyAllWindows()
client.close()

In [3]:
A = open('/Users/winlab/Desktop/stream.mp4','rb').read()
B = open('/Users/winlab/Desktop/Summer.mp4','rb').read()
A[:300000] == B[:300000]

True